# Silver Data Load

In this notebook, we will be filtering, cleaning, and augmenting to create the Silver Layer.

This layer brings the data from various sources into an "Enterprise View," allowing for further analysis and allowing us to answer one of our research questions:

> Do certain intake attributes make an animal more likely to be adopted?

For more information on Medallion Architecture, see [Databricks Glossary](https://www.databricks.com/glossary/medallion-architecture) (Databricks, n.d.).

---

> **Note:** This is under the assumption that you have already loaded the data through the [`notebooks/elt/1_bronze.ipynb`](./1_bronze.ipynb).

### References

Databricks. (n.d.). *Medallion Architecture*. Retrieved May 10, 2025, from [https://www.databricks.com/glossary/medallion-architecture](https://www.databricks.com/glossary/medallion-architecture)

## Checklist

1. **Date Anomalies**  
   - Identify any `intake_date` or `outcome_date` > today’s date  
     - ▢ Flag them with a boolean column (e.g. `is_future_intake`)  
     - ▢ Decide whether to drop or correct (e.g. trim off future records)  
   - Check for missing or null dates  
     - ▢ Log how many, then drop or impute if necessary  
   - Validate date formats & dtypes  
     - ▢ Ensure all date columns are `datetime64`  
     - ▢ Normalize any stray string–dates via `pd.to_datetime(errors='coerce')`

2. **Clean the `age` Column**  
   - Extract number + unit from raw string (e.g. “7 MONTHS”, “2 WEEKS”)  
     - ▢ Write a parser (regex or split) to separate `value` and `unit`  
   - Convert to a consistent numeric metric (`age_years`)  
     - ▢ DAYS → days/365  
     - ▢ WEEKS → (weeks×7)/365  
     - ▢ MONTHS → months/12  
     - ▢ YEARS → years  
   - Handle invalid or missing formats  
     - ▢ Coerce unrecognized strings to `NaN`  
     - ▢ Optionally, add an `age_was_missing` indicator  
   - Add & validate new column  
     - ▢ Create `age_years` in the DataFrame  
     - ▢ Inspect its distribution (`.describe()`, histograms) for outliers

In [ ]:
%pip install -r ../../requirements.txt

Note: you may need to restart the kernel to use updated packages.
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.5/104.5 kB 2.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 15.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.6/254.6 kB 10.0 MB/s eta 0:00:00
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 15.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 18.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.1/111.1 kB 9.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)
Note: you may need to restart the kernel to

In [84]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
pd.set_option('display.max_columns', None)